## Pytorch - Classificateur de textes par Embedding uniquement

### Pandas & Sklearn | Ensembles de données

In [2]:
import pandas as pd
from sklearn.utils import shuffle

In [3]:
df = pd.read_csv(
    "/Users/erwan/Downloads/french_tweets.csv",
)

In [4]:
random_state = 1
shuffled_df = shuffle(df, random_state=random_state)

In [5]:
ds_size = 10000
small_suffled_df = shuffled_df[:ds_size]

### Hugging-Face | Ensembles de données & Tokenizer

In [6]:
import datasets

/opt/anaconda3/envs/IA-torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Transformation du dataframe en base de données Hugging Face

In [7]:
ds = datasets.Dataset.from_pandas(small_suffled_df)
ds

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 10000
})

- Récupération d'un tokenizer Hugging Face entrainé sur un autre notebook

In [18]:
import tokenizers

In [19]:
path = "/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/Only_Embeddings/tokenizers/hf_002.json"

tokenizer = tokenizers.Tokenizer.from_file(path)

In [23]:
res = tokenizer.encode("C'est une jolie maison")
res.tokens

['c', "'", 'est', 'une', 'jolie', 'maison']

### Pytorch - Définition du modèle 

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, drop1, drop2) -> None:
        super(TextClassifier, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = emb_dim
        self.drop1 = drop1
        self.drop2 = drop2

        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.embedding_dim
        )

        self.drop1 = nn.Dropout(drop1)
        self.drop2 = nn.Dropout(drop2)
        self.global_pool1D = nn.AvgPool1d(kernel_size=self.embedding_dim)
        self.linear = nn.Linear(self.embedding_dim, 1)

    def forward(self, x): # x est une séquence déjà "tokenisée"
        x = self.embedding(x)
        x = self.drop1(x)

        # Avg_pool se fait toujours sur la dernière couche avec torch, 
        # pour conserver la dimension de l'embedding on doit changer les 
        # "dimensions" de position.
        z = x.permute(0, 2, 1) 
        z = self.global_pool1D(z)

        z = self.drop2(z)
        res = self.linear(z)

        return res



In [27]:
check_model = TextClassifier(10,100, 0.1, 0.2)

### Fonctions de construction 

In [11]:
from sklearn.model_selection import train_test_split

import datasets                             # Hugging Face, utile pour l'entra
from torch.utils.data import DataLoader

In [28]:
# Pour entraîner un tokenizer voir train_tokenizer (plus haut)

def build_tokenizer(path):
    """
    Prend un chemin vers un fichier .json issu de l'entrainement d'un tokenizer Hugging Face
    """
    tokenizer = Tokenizer.from_pretrained(path)
    
    return tokenizer


# vocab_size sera issu de tokenizer.get_vocab_size()

def build_network(vocab_size, emb_dim, drop1, drop2):
    network = TextClassifier(
        vocab_size=vocab_size,
        emb_dim=emb_dim,
        drop1=drop1,
        drop2=drop2,
    )

    return network


def build_datasets(path, batch_size):
    """
    .csv -> panda -> hugging Face -> DataLoader -> train, val, test
    """
    random_state = 1

    df = pd.read_csv(path)

    raw_learn_ds, raw_test_ds = train_test_split(df, test_size=0.4, random_state=random_state, shuffle=True)
    raw_train_ds, raw_val_ds = train_test_split(raw_learn_ds, test_size=0.2)

    train_ds = 


In [ ]:
"/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/Only_Embeddings/tokenizers/hf_002.json"

### Initialisation de W&B

In [4]:
import wandb
import pprint

In [10]:
wandb.login()

True

- Cloture de la session précédente

In [6]:
wandb.finish()

- Définition des hyperparamètres

In [7]:
sweep_config = dict(
    method='random',
)
parameters_dict = dict(
    vocab_size={'values': [15000]},
    embedding_dim={'values': [50, 100, 150, 200]},
    drop1_layer={'values': [0.2, 0.5]},
    drop2_layer={'value': [0.2]},
    optimizer ={'values': ['adam', 'SGD']},
    batch_size={'values': [32, 64]},
    epoch={'value': [10]}
)

In [8]:
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)


{'method': 'random',
 'parameters': {'batch_size': {'values': [32, 64]},
                'drop1_layer': {'values': [0.2, 0.5]},
                'drop2_layer': {'value': [0.2]},
                'embedding_dim': {'values': [50, 100, 150, 200]},
                'epoch': {'value': [10]},
                'optimizer': {'values': ['adam', 'SGD']},
                'vocab_size': {'values': [15000]}}}


In [9]:
sweep_id = wandb.sweep(sweep_config, project='TextClass_PureEmbedding')

Create sweep with ID: nq73bmkp
Sweep URL: https://wandb.ai/erwanlbv/TextClass_PureEmbedding/sweeps/nq73bmkp


## TESTS

In [49]:
import torch
from torch.utils.data import DataLoader


import datasets
import tokenizers

In [43]:
df = pd.read_csv("/Users/erwan/Downloads/french_tweets.csv")

In [44]:
df

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."
...,...,...
1526719,1,"Oui, cela fonctionne mieux que de l'attendre à..."
1526720,1,Je viens de me réveiller. Ne pas avoir d'école...
1526721,1,Thewdb.com - très cool d'entendre les vieilles...
1526722,1,Êtes-vous prêt pour votre mojo makeover? Deman...


In [34]:
data = torch.from_numpy(df['label'][:5].to_numpy())
data

tensor([0, 0, 0, 0, 0])

In [29]:
df['text'][:5].to_numpy()

array(["- Awww, c'est un bummer. Tu devrais avoir david carr du troisième jour pour le faire. ;ré",
       "Est contrarié qu'il ne puisse pas mettre à jour son facebook en le télémaignant ... et peut-être pleurer en conséquence, l'école aujourd'hui aussi. blabla!",
       "J'ai plongé plusieurs fois pour la balle. A réussi à économiser 50% le reste sort de limites",
       "Tout mon corps a des démangeaisons et comme si c'était en feu",
       'Non, il ne se comporte pas du tout. je suis en colère. pourquoi suis-je ici? Parce que je ne peux pas vous voir partout.'],
      dtype=object)

In [66]:
hf_dataset = datasets.Dataset.from_pandas(df[:100])
hf_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 100
})

In [67]:
for example in hf_dataset:
    print(example)
    break

{'label': 0, 'text': "- Awww, c'est un bummer. Tu devrais avoir david carr du troisième jour pour le faire. ;ré"}


In [92]:
path = '/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/Only_Embeddings/tokenizers/hf_002.json'

tokenizer = tokenizers.Tokenizer.from_file(path)

res = tokenizer.encode("Il était une fois")
res

Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [85]:
padded_res = tokenizer('Il fait beau dehors', padding='max_length', max_length=15)

AttributeError: 'str' object has no attribute 'items'

In [71]:
def tokenize(example):
    res = {
        'label': example['label'],
        'text': tokenizer.encode(example['text']) 
    }
    return res

In [72]:
mapped_hf_dataset = hf_dataset.map(tokenize)

100%|██████████| 100/100 [00:00<00:00, 4003.19ex/s]


In [73]:
torch_ds = mapped_hf_dataset.with_format("torch")

In [74]:
ds = DataLoader(torch_ds, batch_size=10)

for samples, labels in ds:
    print(f"Shape of samples [N, C, H, W]: {samples.shape}")
    print(f"Shape of labels: {labels.shape} {labels.dtype}")
    break

batch_text, batch_label = next(iter(ds))
batch_label, batch_text

RuntimeError: stack expects each tensor to be equal size, but got [24] at entry 0 and [42] at entry 1

In [75]:
for i, batch in enumerate(ds):
    
    print(i, batch)
    if i == 3: break

RuntimeError: stack expects each tensor to be equal size, but got [24] at entry 0 and [42] at entry 1

In [80]:
ds.dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 100
})